# (29) jacobian — bench

**Motivation**: host = ```mach```, device = ```cuda:2``` <br>

In [1]:
# HIDE CODE


import os, sys
from IPython.display import display

# tmp & extras dir
git_dir = os.path.join(os.environ['HOME'], 'Dropbox/git')
extras_dir = os.path.join(git_dir, 'jb-vae/_extras')
fig_base_dir = os.path.join(git_dir, 'jb-vae/figs')
tmp_dir = os.path.join(git_dir, 'jb-vae/tmp')

# GitHub
sys.path.insert(0, os.path.join(git_dir, '_IterativeVAE'))
from figures.fighelper import *
from vae.train_vae import *

# warnings, tqdm, & style
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=UserWarning)
from rich.jupyter import print
%matplotlib inline
set_style()

In [2]:
from base.utils_model import load_quick

device_idx = 2
device = f'cuda:{device_idx}'

In [3]:
def get_perf(first, first_descriptor, second, second_descriptor):
    if first.times[0] < second.times[0]:
        winner = first_descriptor
        faster = first.times[0]
        slower = second.times[0]
    else:
        winner = second_descriptor
        faster = second.times[0]
        slower = first.times[0]
    
    gain = 100 * (slower - faster) / slower
    print(f"Performance delta: {gain:.2f} percent improvement with '{winner}'")

## jacrev and jacfwd

In [5]:
def predict(x):
    return F.linear(x, weight, bias).tanh()

In [6]:
batch_size = 123
n_latents = 16
n_pix = 256

weight = torch.randn(n_pix, n_latents)
print(f"weight shape = {weight.shape}")

bias = torch.randn(n_pix)
z = torch.randn(batch_size, n_latents)

weight shape = torch.Size([256, 16])

In [7]:
compute_batch_jacobian = torch.func.vmap(
    func=torch.func.jacfwd(predict),
    in_dims=0,
)
jacobian = compute_batch_jacobian(z)
jacobian.shape

torch.Size([123, 256, 16])

In [9]:
from torch.utils.benchmark import Timer

stmt_fwd = "torch.func.vmap(torch.func.jacfwd(predict, argnums=0), in_dims=0)(z)"
stmt_rev = "torch.func.vmap(torch.func.jacrev(predict, argnums=0), in_dims=0)(z)"
timer_fwd = Timer(stmt=stmt_fwd, globals=globals())
timer_rev = Timer(stmt=stmt_rev, globals=globals())

fwd_times = timer_fwd.timeit(500)
rev_times = timer_rev.timeit(500)

print(fwd_times)
print(rev_times)

<torch.utils.benchmark.utils.common.Measurement object at 0x7db20e1a0610>
torch.func.vmap(torch.func.jacfwd(predict, argnums=0), in_dims=0)(z)
  1.93 ms
  1 measurement, 500 runs , 1 thread

<torch.utils.benchmark.utils.common.Measurement object at 0x7db1046404d0>
torch.func.vmap(torch.func.jacrev(predict, argnums=0), in_dims=0)(z)
  17.99 ms
  1 measurement, 500 runs , 1 thread

In [13]:
print(f"latent dim: {n_latents}  ———  num pixels: {n_pix}")
get_perf(fwd_times, 'jacfwd', rev_times, 'jacrev')

latent dim: 16  ———  num pixels: 256

Performance delta: 89.27 percent improvement with 'jacfwd'

In [15]:
batch_size = 123
n_latents = 512
n_pix = 256

weight = torch.randn(n_pix, n_latents)
bias = torch.randn(n_pix)
z = torch.randn(batch_size, n_latents)

In [16]:
from torch.utils.benchmark import Timer

stmt_fwd = "torch.func.vmap(torch.func.jacfwd(predict, argnums=0), in_dims=0)(z)"
stmt_rev = "torch.func.vmap(torch.func.jacrev(predict, argnums=0), in_dims=0)(z)"
timer_fwd = Timer(stmt=stmt_fwd, globals=globals())
timer_rev = Timer(stmt=stmt_rev, globals=globals())

fwd_times = timer_fwd.timeit(500)
rev_times = timer_rev.timeit(500)

print(fwd_times)
print(rev_times)

<torch.utils.benchmark.utils.common.Measurement object at 0x7db10290c990>
torch.func.vmap(torch.func.jacfwd(predict, argnums=0), in_dims=0)(z)
  238.01 ms
  1 measurement, 500 runs , 1 thread

<torch.utils.benchmark.utils.common.Measurement object at 0x7db0e833f350>
torch.func.vmap(torch.func.jacrev(predict, argnums=0), in_dims=0)(z)
  81.40 ms
  1 measurement, 500 runs , 1 thread

In [17]:
print(f"latent dim: {n_latents}  ———  num pixels: {n_pix}")
get_perf(fwd_times, 'jacfwd', rev_times, 'jacrev')

latent dim: 512  ———  num pixels: 256

Performance delta: 65.80 percent improvement with 'jacrev'

## Explore with real IP-VAE

### (option 1) latent_dim < n_pix

In [4]:
model_type = 'poisson'
cfg_vae, cfg_tr = default_configs('vH16', model_type, 'jac|mlp')

cfg_vae['n_latents'] = [100]
cfg_vae['init_scale'] = 1e-2
cfg_vae['seq_len'] = 20

In [5]:
vae = HIPVAE(CFG_CLASSES[model_type](**cfg_vae))
tr = TrainerVAE(vae, ConfigTrainVAE(**cfg_tr), device=device)

In [6]:
x = next(iter(tr.dl_vld))[0]
x = x.flatten(start_dim=1)

self = tr.model.input_layer
self.reset_state(len(x))

In [7]:
dist, spks, pred = self.generate(0.2, True)
residual = x - pred

In [8]:
print(self.dim < self._cfg.input_sz ** 2)

True

In [9]:
func = torch.func.jacrev(func=self.decode, argnums=0)
jacobian = torch.func.vmap(func=func, in_dims=0, chunk_size=None)

In [10]:
from torch.utils.benchmark import Timer

stmt_main = "self.encode(residual, spks)"
stmt_alt = "torch.einsum('bmk, bm -> bk', jacobian(spks).squeeze(), x)"
timer_main = Timer(stmt=stmt_main, globals=globals())
timer_alt = Timer(stmt=stmt_alt, globals=globals())

timer_main = timer_main.timeit(500)
timer_alt = timer_alt.timeit(500)

print(timer_main)
print(timer_alt)

<torch.utils.benchmark.utils.common.Measurement object at 0x7c27b5f7ebd0>
self.encode(residual, spks)
  2.09 ms
  1 measurement, 500 runs , 1 thread

<torch.utils.benchmark.utils.common.Measurement object at 0x7c27c521c150>
torch.einsum('bmk, bm -> bk', jacobian(spks).squeeze(), x)
  3.77 ms
  1 measurement, 500 runs , 1 thread

In [11]:
get_perf(timer_main, 'main', timer_alt, 'alt')

Performance delta: 44.65 percent improvement with 'main'

### (option 2) latent_dim > n_pix

In [12]:
model_type = 'poisson'
cfg_vae, cfg_tr = default_configs('vH16', model_type, 'jac|mlp')

cfg_vae['n_latents'] = [512]
cfg_vae['init_scale'] = 1e-2
cfg_vae['seq_len'] = 20

In [13]:
vae = HIPVAE(CFG_CLASSES[model_type](**cfg_vae))
tr = TrainerVAE(vae, ConfigTrainVAE(**cfg_tr), device=device)

In [14]:
x = next(iter(tr.dl_vld))[0]
x = x.flatten(start_dim=1)

self = tr.model.input_layer
self.reset_state(len(x))

In [15]:
dist, spks, pred = self.generate(0.2, True)
residual = x - pred

In [16]:
print(self.dim < self._cfg.input_sz ** 2)

False

In [17]:
func = torch.func.jacfwd(func=self.decode, argnums=0)
jacobian = torch.func.vmap(func=func, in_dims=0, chunk_size=None)

In [18]:
from torch.utils.benchmark import Timer

stmt_main = "self.encode(residual, spks)"
stmt_alt = "torch.einsum('bmk, bm -> bk', jacobian(spks).squeeze(), x)"
timer_main = Timer(stmt=stmt_main, globals=globals())
timer_alt = Timer(stmt=stmt_alt, globals=globals())

timer_main = timer_main.timeit(500)
timer_alt = timer_alt.timeit(500)

print(timer_main)
print(timer_alt)

<torch.utils.benchmark.utils.common.Measurement object at 0x7c27c2fcd290>
self.encode(residual, spks)
  11.23 ms
  1 measurement, 500 runs , 1 thread

<torch.utils.benchmark.utils.common.Measurement object at 0x7c27c5b765d0>
torch.einsum('bmk, bm -> bk', jacobian(spks).squeeze(), x)
  41.12 ms
  1 measurement, 500 runs , 1 thread

In [19]:
get_perf(timer_main, 'main', timer_alt, 'alt')

Performance delta: 72.69 percent improvement with 'main'

Performance delta: 44.65 percent improvement with 'main'

## When implementinc mlp decoder

In [54]:
for i in range(5):
    print(_build_mlp_dec(100, 784, 'swish', i))

Sequential(
  (0): Linear(in_features=100, out_features=784, bias=True)
)

Sequential(
  (0): Linear(in_features=100, out_features=280, bias=True)
  (1): SiLU(inplace=True)
  (2): Linear(in_features=280, out_features=784, bias=True)
)

Sequential(
  (0): Linear(in_features=100, out_features=198, bias=True)
  (1): SiLU(inplace=True)
  (2): Linear(in_features=198, out_features=393, bias=True)
  (3): SiLU(inplace=True)
  (4): Linear(in_features=393, out_features=784, bias=True)
)

Sequential(
  (0): Linear(in_features=100, out_features=167, bias=True)
  (1): SiLU(inplace=True)
  (2): Linear(in_features=167, out_features=279, bias=True)
  (3): SiLU(inplace=True)
  (4): Linear(in_features=279, out_features=466, bias=True)
  (5): SiLU(inplace=True)
  (6): Linear(in_features=466, out_features=784, bias=True)
)

Sequential(
  (0): Linear(in_features=100, out_features=150, bias=True)
  (1): SiLU(inplace=True)
  (2): Linear(in_features=150, out_features=226, bias=True)
  (3): SiLU(inplace=True)
  (4): Linear(in_features=226, out_features=341, bias=True)
  (5): SiLU(inplace=True)
  (6): Linear(in_features=341, out_features=514, bias=True)
  (7): SiLU(inplace=True)
  (8): Linear(in_features=514, out_features=784, bias=True)
)